In [0]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt


transformations = transforms.Compose([transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST('fill_your_working_directory', train=True, transform=transformations, download=True)
mnist_test = torchvision.datasets.MNIST('fill_your_working_directory', train=False, transform=transformations)

In [0]:
print(len(mnist_train))
print(len(mnist_train[0]))
mnist_train

In [0]:
print(len(mnist_test))
print(len(mnist_test[0]))
mnist_test

In [0]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = 32, shuffle = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = 32, shuffle = True)

In [0]:
class OneLayerModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(OneLayerModel, self).__init__()
        self.h1 = nn.Linear(input_dim,output_dim)
    
        
    def forward(self, x):
        output = x.view(-1,28*28)
        output = self.h1(output)
        return output

In [0]:
model = OneLayerModel(28*28,10)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

In [0]:
def train(model, train_loader, val_loader, loss_func, opt,num_epochs=10):
    train_loss_list = []
    training_accuracy = []
    test_loss_list = []
    testing_accuracy = []
    for e in range(num_epochs):
        model.train()
        train_loss = 0
        num_correct = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            opt.zero_grad()
            output = model(data)
            temp = loss_func(output, target)
            train_loss += temp.item()
            temp.backward()
            opt.step()
            pred = output.data.max(1, keepdim=True)[1]
            target = target.view_as(pred)
            num_correct += sum(pred==target).item()
        train_loss_list.append(train_loss/len(train_loader))
        accuracy = num_correct/len(train_loader.dataset)
        training_accuracy.append(accuracy) 


        model.eval()
        test_loss = 0
        correct = 0.0
        for data, target in val_loader:
            data, target = Variable(data), Variable(target)
            output = model(data)

            test_loss += loss_func(output, target).item()

            pred = output.data.max(1, keepdim=True)[1]
            target = target.view_as(pred)
            correct += sum(pred==target).item()

        test_loss_list.append(test_loss/len(test_loader))
        accuracy = correct/len(val_loader.dataset)
        testing_accuracy.append(accuracy)
    return train_loss_list, training_accuracy, test_loss_list, testing_accuracy
      
     

In [0]:
train_loss_list, training_accuracy, test_loss_list, testing_accuracy = train(model, train_loader, test_loader, loss, optimizer, 15)

In [0]:
import torch.nn.functional as F
class TwoLayerModel(nn.Module):
   
    def __init__(self):
        super(TwoLayerModel, self).__init__()
        self.l1 = nn.Linear(784,520)
        self.l2 = nn.Linear(520, 10)
    def forward(self,x):
        x = x.view(-1, 784) 
        x = F.relu(self.l1(x))
        return self.l2(x)
    

In [0]:
model2 = TwoLayerModel()
loss2 = nn.CrossEntropyLoss()
optimizer2 = optim.SGD(model2.parameters(), lr = 0.001, momentum=0.9)

In [0]:
train_loss_list2, training_accuracy2, test_loss_list2, testing_accuracy2 = train(model2, train_loader, test_loader, loss2, optimizer2, 15)